In [6]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [7]:
cancer = load_breast_cancer() # 유방암 데이터

In [8]:
type(cancer)

sklearn.utils._bunch.Bunch

In [9]:
# Bunch 타입이라는게 뭐임? : df 랑 비슷한 구조

In [10]:
cancer["data"].shape # data 라는 키로 접근가능~~~

(569, 30)

In [11]:
cancer["data"][0]

array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
       3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
       8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
       3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
       1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01])

In [12]:
# 한명에 대한 30개의 데이터

- 데이터 모양 확인
- 변수가 30개임~~

In [13]:
cancer["target"][:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

- 종속변수
    - 0 : 악성
    - 1 : 양성

In [14]:
cancer["feature_names"] # 변수명이 드가있다

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [15]:
len(cancer["feature_names"])

30

In [16]:
# 데이터 프레임으로 만들어?

In [17]:
cancer["target_names"]

array(['malignant', 'benign'], dtype='<U9')

# 데이터 분할

In [18]:
x_train, x_test, y_train, y_test = train_test_split(
cancer.data,
cancer.target,
test_size = 0.25,
stratify = cancer.target, random_state = 33)

# 모델 학습

In [19]:
clf = SVC()
clf.fit(x_train, y_train)

SVC()

In [20]:
pred = clf.predict(x_test)

In [21]:
accuracy_score(y_test, pred) # 성능이 좋다!

0.9230769230769231

# 최적화

In [22]:
mm = MinMaxScaler()
scaled_train = mm.fit_transform(x_train)
scaled_test = mm.transform(x_test)

In [23]:
clf2 = SVC()

In [24]:
clf2.fit(scaled_train, y_train)

SVC()

In [25]:
clf2.score(scaled_test, y_test)

0.972027972027972

# 하이퍼파라미터 튜닝

- 머신러닝 모형이 완성되고 나서 매개변수 최적화를 통해서 예측성능을 더 극대화

In [26]:
param_grid = {"C": [0.0001, 0.001, 0.01, 0.1,1,10,100,1000],
               "gamma" : [0.0001, 0.001, 0.01, 0.1,1,10,100,1000]
              }

grid = GridSearchCV(SVC(), param_grid = param_grid, cv = 5, n_jobs = -1)

In [27]:
grid.fit(scaled_train, y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [28]:
grid.best_score_

np.float64(0.983529411764706)

In [29]:
grid.best_params_

{'C': 10, 'gamma': 1}

In [30]:
grid.score(scaled_test, y_test)

0.965034965034965

# 파이프라인 구축

- 여러 알고리즘 비교

- 전체 공정을 시퀀스로 연결하여 구성

## 파이프라인 구축 예

- case 1 : 표준 방법
    - 작업 순서를 나열하고, 이름을 각각 부여
    - 후보가 유력한 알고리즘과 전처리기를 배치

In [31]:
pipe = Pipeline([("scaler", MinMaxScaler()), ("classifier", SVC())])

In [32]:
pipe

Pipeline(steps=[('scaler', MinMaxScaler()), ('classifier', SVC())])

- case2 : 간소화 방법
- 이름을 부여하지 않고 자동으로 부여됨

In [34]:
pipe_sim = make_pipeline(MinMaxScaler(), SVC())
pipe_sim

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('svc', SVC())])

# 파이프라인 적용

In [35]:
pipe

Pipeline(steps=[('scaler', MinMaxScaler()), ('classifier', SVC())])

In [39]:
# 하이퍼 파라미터 정의

param_grid = [
    {
        "classifier" : [ SVC() ],
        "scaler" : [StandardScaler(), MinMaxScaler()],
        # 이름__파라미터명 => 해당 알고리즘에 적용되는 파라미터
        "classifier__C" : [0.001, 0.01, 0.1, 1, 10, 100],
        "classifier__gamma" : [0.001, 0.01, 0.1, 1, 10, 100],
    },
    {
        "classifier" : [RandomForestClassifier(random_state = 33)],
        "scaler" : [None],
        "classifier__max_features" : [1, 2, 3]
    }
]
# 75가지 경우의 수...

In [38]:
# 결합(파이프라인, 파이퍼 파라미터 튜닝)
grid = GridSearchCV(pipe, param_grid, cv = 5, n_jobs = -1)

In [40]:
grid

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier', SVC())]),
             n_jobs=-1,
             param_grid=[{'classifier': [SVC()],
                          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'scaler': [StandardScaler(), MinMaxScaler()]},
                         {'classifier': [RandomForestClassifier(random_state=33)],
                          'classifier__max_features': [1, 2, 3],
                          'scaler': [None]}])

In [42]:
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier', SVC())]),
             n_jobs=-1,
             param_grid=[{'classifier': [SVC()],
                          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'scaler': [StandardScaler(), MinMaxScaler()]},
                         {'classifier': [RandomForestClassifier(random_state=33)],
                          'classifier__max_features': [1, 2, 3],
                          'scaler': [None]}])

In [43]:
grid.best_score_

np.float64(0.9811764705882353)

#### 성능좋은거 출력함

In [44]:
grid.best_params_

{'classifier': SVC(),
 'classifier__C': 10,
 'classifier__gamma': 0.01,
 'scaler': StandardScaler()}

In [45]:
grid.predict(x_test)

array([1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1])

In [46]:
grid.score(x_test, y_test)

0.972027972027972